<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="../assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Autonomous Alpha Agents</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

### Week 6 Day 4

And now - introducing the Capstone project:


# Autonomous Alpha Agents

An equity trading simulation, with 3 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

The goal of today's lab is to make a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please do not use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [ ]:
import os
from dotenv import load_dotenv
from alpha_client import get_stock_tools_openai
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

### Let's start by gathering the params for all our MCP servers

In [2]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-memory"]},
]

trader_tools = []

In [3]:
if os.getenv("FINANCIAL_DATASETS_API_KEY"):
    directory = "/Users/ed/projects/mcp-server"
    env = {"FINANCIAL_DATASETS_API_KEY": os.getenv("FINANCIAL_DATASETS_API_KEY")}
    trader_mcp_server_params.append({"command": "uv", "args": ["--directory", directory, "run", "server.py"], "env": env})
else:
    trader_tools += await get_stock_tools_openai()

In [4]:
researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### Now create the MCPServerStdio for each

In [6]:
researcher_mcp_servers = [MCPServerStdio(params, cache_tools_list=True) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, cache_tools_list=True) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [7]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4o-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [8]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [ ]:


for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, "What's the latest news on Amazon?")
display(Markdown(result.final_output))



### Look at the trace

https://platform.openai.com/traces

In [ ]:
alice_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Alice").reset(alice_initial_strategy)

display(Markdown(await read_accounts_resource("Alice")))
display(Markdown(await read_strategy_resource("Alice")))


### And now - to create our Trader Agent

In [11]:
agent_name = "Alice"

account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [ ]:
display(Markdown(instructions))

### And to run our Trader

In [ ]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name="Alice",
    instructions=instructions,
    tools=trader_tools + [researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace("Alice"):
    result = await Runner.run(trader, prompt)
display(Markdown(result.final_output))

### Then go and look at the trace

http://platform.openai.com/traces


In [ ]:
await read_accounts_resource(agent_name)

### Now it's time to review the Python module made from this:

`traders.py`

Note that if you're using free market data via Alpha Vantage, you'll need to update a constant in this module.

Change the code near the very top to the directory containing your mcp-server from financial datasets:

`DIRECTORY_FOR_PAID_MARKET_DATA_MCP_SERVER = "/Users/ed/projects/mcp-server"`


In [1]:
from traders import Trader


In [2]:
trader = Trader("Alice")

In [3]:
await trader.init_agent()

Agent(name='Alice', instructions="\nYou are Alice, a trader on the stock market. Your account is under your name, Alice.\nYou actively manage your portfolio according to your strategy.\nYou have access to tools including a researcher to research online for news and opportunities, based on your request.\nYou also have tools to access to the latest financial data for stocks, via a financial data API.\nAnd you have tools to buy and sell stocks using your account name Alice.\nYou can use your entity tools as a persistent memory to store and recall information; you share\nthis memory with other traders and can benefit from the group's knowledge.\nUse these tools to carry out research, make decisions, and execute trades.\nAfter you've completed trading, send a push notification with a brief summary of activity, then reply with a 2-3 sentence appraisal.\nYour goal is to maximize your profits according to your strategy.\n", handoff_description=None, handoffs=[], model='gpt-4o-mini', model_sett

In [4]:
result = await trader.run()

Errored MCP tool result: meta=None content=[] isError=False


In [ ]:
display(Markdown(result))

### Now look at the trace

https://platform.openai.com/traces